In [ ]:
# Copyright 2022 Google LLC.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 9. Audience Upload to GMP

[GMP and Google Ads Connector](https://github.com/GoogleCloudPlatform/cloud-for-marketing/tree/main/marketing-analytics/activation/gmp-googleads-connector#gmp-and-google-ads-connector) is used to upload audience data to GMP (e.g. Google Analytics, Campaign Manager) or Google Ads in an automatic and reliable way.

Following sections provide high level guidelines on deploying and configuring [GMP and Google Ads Connector](https://github.com/GoogleCloudPlatform/cloud-for-marketing/tree/main/marketing-analytics/activation/gmp-googleads-connector#gmp-and-google-ads-connector). For detailed instructions on how to set up different GMP endpoints, refer to solution's [README.md](https://github.com/GoogleCloudPlatform/cloud-for-marketing/blob/main/marketing-analytics/activation/gmp-googleads-connector/README.md).

## Requirements

This notebook requires BigQuery table containing scored audience list. Refer to [7.batch_scoring.ipynb](7.batch_scoring.ipynb) for details on how to get scored audience.

## Import required modules

In [ ]:
# Add custom utils module to Python environment
import os
import sys
sys.path.append(os.path.abspath(os.pardir))

from IPython import display

from utils import helpers

## Deploy GMP and Google Ads Connector

First clone the source code by executing below cell:

In [ ]:
!git clone https://github.com/GoogleCloudPlatform/cloud-for-marketing.git ~/cloud-for-marketing

Next, exectute following two steps to deploy GMP and Google Ads Connector on your GCP project.

1. Copy following content:

```bash
cd ~/cloud-for-marketing/marketing-analytics/activation/gmp-googleads-connector && ./deploy.sh default_install
```

2. Execute following cell to start a new **Terminal session** and paste above copied content to the Terminal. **NOTE:** This notebook uses [Google Analytics Measurement Protocol API](https://developers.google.com/analytics/devguides/collection/protocol/v1/devguide) to demonstrate audience upload, thus choose `0` on **Step 5: Confirm the integration with external APIs...** during the installation process on the Terminal session.

It takes about 3 minutes to setup audience uploader pipeline.

In [ ]:
display.HTML('<a href="" data-commandlinker-command="terminal:create-new">▶Access Terminal◀︎</a>')

When the deployment is done, you can verify three Cloud Functions deployments via the [Cloud Console UI](https://console.cloud.google.com/functions). If deployment is succeeded, move to next section to upload audience data to Google Analytics via JSONL file.

## Configure audience upload endpoint

Different audience upload endpoint APIs have different configurations. Following demonstrates how endpoint for Google Analytics can be configured via Measurement Protocol. Refer to [3.3. Configurations of APIs](https://github.com/GoogleCloudPlatform/cloud-for-marketing/tree/main/marketing-analytics/activation/gmp-googleads-connector#33-configurations-of-apis) for detailed configuration options for other endpoints.

Update following GA values according to your needs in the following cell. Refer to [Working with the Measurement Protocol](https://developers.google.com/analytics/devguides/collection/protocol/v1/devguide) for details on field names and correct values.

```json
{
    "t": "event",
    "ec": "video",
    "ea": "play",
    "ni": "1",
    "tid": "UA-112752759-1"
}
```

In [ ]:
%%writefile cloud-for-marketing/marketing-analytics/activation/gmp-googleads-connector/config_api.json
{
  "MP": {
    "default": {
      "mpConfig": {
        "v": "1",
        "t": "event",
        "ec": "video",
        "ea": "play",
        "ni": "1",
        "tid": "UA-XXXXXXXXX-Y"
      }
    }
  }
}

## Create audience list JSON files

**GMP and Google Ads Connector**'s [Google Analytics Measurement Protocol pipeline](https://github.com/GoogleCloudPlatform/cloud-for-marketing/blob/main/marketing-analytics/activation/gmp-googleads-connector/README.md#41-mp-google-analytics-measurement-protocol) requires [JSONL](https://jsonlines.org) text format. Following cells help to export BigQuery table containing audience list as `JSONL` file to Google Cloud Storage Bucket. **NOTE:** This solution has specific file naming requirement to work properly. Refer to [3.4. Name convention of data files](https://github.com/GoogleCloudPlatform/cloud-for-marketing/blob/main/marketing-analytics/activation/gmp-googleads-connector/README.md#34-name-convention-of-data-files) for more details.


As soon as the file is uploaded, **GMP and Google Ads Connector** processes it and sends it via Measurement Protocol to Google Analytics property configured above (`"tid": "UA-XXXXXXXXX-Y"`).

In [ ]:
configs = helpers.get_configs('config.yaml')
dest_configs = configs.destination

# GCP project ID
PROJECT_ID = dest_configs.project_id
# Name of BigQuery dataset
DATASET_NAME = dest_configs.dataset_name

In [ ]:

# Google Cloud Storage Bucket name to store audience upload JSON files
# NOTE: The name should be same as indicated while deploying
# "GMP and Google Ads Connector" on the Terminal
GCS_BUCKET = 'bucket'
# This Cloud Storage folder is monitored by the "GMP and Google Ads Connector"
# to send over to endpoint (eg: Google Analytics).
GCS_FOLDER = 'outbound'
# File name to export BigQuery Table to Cloud Storage
JSONL_FILENAME = 'myproject_API[MP]_config[default].jsonl'
# BigQuery table containing scored audience data
AUDIENCE_SCORE_TABLE_NAME = 'table'

In [ ]:
%%bash -s $PROJECT_ID $DATASET_NAME $AUDIENCE_SCORE_TABLE_NAME $GCS_BUCKET $GCS_FOLDER $JSONL_FILENAME

bq extract \
    --destination_format NEWLINE_DELIMITED_JSON \
    $1:$2.$3 \
    gs://$4/$5/$6

## Next

If schedule-based automated pipeline is required, then refer to [10.automatyed_activation_pipeline.ipynb](10.automatyed_activation_pipeline.ipynb) which helps to automate following steps:
1. Generate prediction dataset
2. Batch score the audience
3. Upload audience to GMP endpoint